In [ ]:
cd /content/drive/MyDrive/lusip

/content/drive/MyDrive/lusip


In [ ]:
!ls

 AccVal_acc.png      checkpoint         model_apple.h5
 apple		     data	       'new plant diseases dataset(augmented)'
 apple_app1.tflite   heatmap.ipynb     'New Plant Diseases Dataset(Augmented)'
 apple_app.tflite    lables.txt         try.ipynb
 apple_mobilnet      LossVal_loss.png


In [ ]:
from glob import glob
import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('apple_mobilnet')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [ ]:
for layer in model.layers:
  print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f44b5fe1ed0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f44b5fde890> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f44b5e4d090> False
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7f44b5fd72d0> False
<tensorflow.python.keras.layers.convolutional.DepthwiseConv2D object at 0x7f44b5e4d0d0> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f44b5fd7c90> False
<tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7f44b1ea3a90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f44b1eab650> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f44b1eac890> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f44b1eb2c90> False
<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7f44

In [ ]:
model.trainable = True

In [ ]:
print("Number of layers in the base model: ", len(model.layers))

Number of layers in the base model:  157


In [ ]:
fine=100
for layer in model.layers[:fine]:
  layer.trainable =  False

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer = tf.keras.optimizers.RMSprop(lr=0.0001/10),
  metrics=['accuracy']
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [ ]:
len(model.trainable_variables)

56

In [ ]:
train_path = '/content/drive/MyDrive/lusip/New Plant Diseases Dataset(Augmented)/train'
valid_path = '/content/drive/MyDrive/lusip/New Plant Diseases Dataset(Augmented)/valid'

In [ ]:
folders = glob('/content/drive/MyDrive/lusip/New Plant Diseases Dataset(Augmented)/train/*')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/lusip/New Plant Diseases Dataset(Augmented)/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 7771 images belonging to 4 classes.


In [ ]:

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/lusip/New Plant Diseases Dataset(Augmented)/valid',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1943 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
cp_callbacks= ModelCheckpoint(filepath='./',
                              save_weights_only=True,
                              save_best_only=True)
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=[cp_callbacks]
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
243/243 [==============================] - 589s 2s/step - loss: 0.0045 - accuracy: 0.9988 - val_loss: 0.0089 - val_accuracy: 0.9969
Epoch 2/10
243/243 [==============================] - 588s 2s/step - loss: 0.0042 - accuracy: 0.9987 - val_loss: 0.0083 - val_accuracy: 0.9969
Epoch 3/10
243/243 [==============================] - 590s 2s/step - loss: 0.0044 - accuracy: 0.9985 - val_loss: 0.0073 - val_accuracy: 0.9974
Epoch 4/10
243/243 [==============================] - 583s 2s/step - loss: 0.0036 - accuracy: 0.9987 - val_loss: 0.0062 - val_accuracy: 0.9974
Epoch 5/10
243/243 [==============================] - 586s 2s/step - loss: 0.0032 - accuracy: 0.9992 - val_loss: 0.0051 - val_accuracy: 0.9974
Epoch 6/10
243/243 [==============================] - 586s 2s/step - loss: 0.0058 - accuracy: 0.9985 - val_loss: 0.0045 - val_accuracy: 0.9979
Epoch 7/10
243/243 [==============================] - 584s 2s/step - loss: 0.0039 - accuracy: 0.9988 - val_loss: 0.0034 - val_accuracy: 0.9979

In [ ]:
model.save('apple_finetune')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: apple_finetune/assets


In [1]:
from tensorflow.keras.models import load_model
model.save('apple_ft.h5')

NameError: ignored